In [1]:
!pip install setfit

In [8]:

from datasets import load_dataset
from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitModel, SetFitTrainer

from datasets import Dataset
import numpy as np

In [80]:
class OneHotEncoder:
    def __init__(self):
        self.unique_labels = None

    def fit_transform(self, labels):
        self.unique_labels = list(set(labels))
        self.unique_labels.sort()
        encoded_labels = []

        for label in labels:
            encoded_label = [0] * len(self.unique_labels)
            index = self.unique_labels.index(label)
            encoded_label[index] = 1
            encoded_labels.append(encoded_label)

        return encoded_labels

    def inverse_transform(self, encoded_labels):
        original_labels = []

        for encoded_label in encoded_labels:
            try:
                index = encoded_label.index(1)
                original_labels.append(self.unique_labels[index])
            except ValueError:
                original_labels.append(None)

        return original_labels

In [81]:
texts = ["I want to make an in-store purchase", "I will buy from store","make a purchase in person",
         "make a payment","redeem rewards","transaction declined","can i pay at store","i like eating momos"]
labels = ['purchase','purchase','purchase','other','other','other','other','food']

In [82]:
enc = OneHotEncoder()
enc_labels = enc.fit_transform(labels)

In [83]:
# Create a dictionary with 'text' and 'label' keys
data = {"text": texts, "label": enc_labels}

# Create a Dataset object from the dictionary
custom_dataset = Dataset.from_dict(data)

# Print the dataset
print(custom_dataset)

Dataset({
    features: ['text', 'label'],
    num_rows: 8
})


In [84]:
train_ds = custom_dataset
test_ds = custom_dataset

In [85]:
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2",multi_target_strategy="one-vs-rest")

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [86]:
# Create trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    loss_class=CosineSimilarityLoss,
    batch_size=16,
    num_iterations=20, # Number of text pairs to generate for contrastive learning
    num_epochs=1 # Number of epochs to use for contrastive learning
)

<ipython-input-86-10f76729619e>:2: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(


Map:   0%|          | 0/8 [00:00<?, ? examples/s]

In [87]:
trainer.train()

***** Running training *****
  Num unique pairs = 320
  Batch size = 16
  Num epochs = 1
  Total optimization steps = 20


Step,Training Loss


In [88]:
y=model.predict(["I will buy it from the Gap store myself","I will pay at the store","I want to redeem rewards",
             "help me in reedem store rewards","i will buy a new shirt from old navy","change atm pin","i love eating","hello how are you"],  )
y

tensor([[0, 0, 1],
        [0, 1, 0],
        [0, 1, 0],
        [0, 1, 0],
        [0, 0, 1],
        [0, 1, 0],
        [1, 0, 0],
        [0, 0, 0]])

In [89]:
enc.inverse_transform(y.tolist())

['purchase', 'other', 'other', 'other', 'purchase', 'other', 'food', None]

In [90]:
p=model.predict_proba(["I will buy it from the Gap store myself","I will pay at the store","I want to redeem rewards",
             "help me in reedem store rewards","i will buy a new shirt from old navy","change atm pin","i love eating","hello how are you"]).tolist()


In [75]:
p

[[0.04597599365509583, 0.0891492184822894, 0.8421230014204194],
 [0.031052715604345204, 0.9160642655437964, 0.05565579991813705],
 [0.03440266731172787, 0.9090784717382613, 0.05569902442441118],
 [0.03080396977908401, 0.8119346230993881, 0.13386681828024724],
 [0.06757990365414174, 0.13110292362428458, 0.7057893399341212],
 [0.04138557439656298, 0.8566767030955524, 0.0780118434618092],
 [0.5728095817279562, 0.15774779075272566, 0.13291106236632347],
 [0.14606153839382857, 0.31948772167073564, 0.2614867568412487]]

In [72]:
[sum(i) for i in p]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[0.977248184167003,
 1.0027727798716157,
 0.9991801647251548,
 0.9766054048558555,
 0.9044721945083509,
 0.9760741261366107,
 0.8634684186426794]